# Loop (for hyperparameter tuning)

## get mem1, 2 with no cost

In [ ]:
RLforFF.env_kwargs_for_curriculum_training

{'dt': 0.1,
 
 'dv_cost_factor': 50,
 'dw_cost_factor': 50,
 'w_cost_factor': 50,
 'print_ff_capture_incidents': False,
 'print_episode_reward_rates': True,
 'flash_on_interval': 0.3,
 'max_in_memory_time': 3,
 'num_obs_ff': 2}

In [ ]:
import copy
max_in_memory_time = 3
params = {
          'dv_cost_factor': 0,
          'dw_cost_factor': 0,
          'w_cost_factor': 0,
}

for num_obs_ff in [2, 1]:
    overall_folder = f'multiff_analysis/RL_models/SB3_stored_models/all_agents/env1_relu/ff{num_obs_ff}/'

    env_kwargs = {'num_obs_ff': num_obs_ff,
                'max_in_memory_time': max_in_memory_time,
                'print_ff_capture_incidents': False
                #'reward_per_ff': 120,
    }

    # check if num_obs_ff is consistent with the name in overall_folder. If not, raise an error
    if not f'ff{env_kwargs["num_obs_ff"]}' in overall_folder:
        raise ValueError('num_obs_ff is not consistent with the name in overall_folder')

    RLforFF = sb3_class.SB3forMultifirefly(**params, overall_folder=overall_folder, **env_kwargs)
    RLforFF.make_env()
    RLforFF.make_agent()
    RLforFF.env_kwargs_for_curriculum_training = copy.deepcopy(RLforFF.env_kwargs)
    RLforFF._make_agent_for_curriculum_training()
    RLforFF.load_best_model_postcurriculum(load_replay_buffer=True)
    RLforFF.regular_training(best_model_save_path=RLforFF.best_model_postcurriculum_dir, env_params_to_save=RLforFF.env_kwargs_for_curriculum_training)



Firely capture rate for the episode:  98 ff for 102.49999999999845 s: -------------------> 0.96
Total reward for the episode:  9800.0
Cost breakdown:  {'time_cost': 0.0, 'dv_cost': 0.0, 'dw_cost': 0.0, 'w_cost': 0.0}
TIME before resetting: 102.49999999999845
current linear_terminal_vel:  0.01
current angular_terminal_vel:  0.01
current dt:  0.1
current full_memory:  30
current time_cost:  0
current dv_cost_factor:  0
current dw_cost_factor:  0
current w_cost_factor:  0
current distance2center_cost:  0
current flash_on_interval:  0.3
current num_obs_ff:  1
current max_in_memory_time:  3

 episode:  289
Firely capture rate for the episode:  100 ff for 102.49999999999845 s: -------------------> 0.98
Total reward for the episode:  10000.0
Cost breakdown:  {'time_cost': 0.0, 'dv_cost': 0.0, 'dw_cost': 0.0, 'w_cost': 0.0}
TIME before resetting: 102.49999999999845
current linear_terminal_vel:  0.01
current angular_terminal_vel:  0.01
current dt:  0.1
current full_memory:  30
current time_cost

## retrain (Resume training) best model from ff1 and ff2, make sure it's mem3...

In [ ]:
for max_in_memory_time in [2, 1, 3]:
    for num_obs_ff in [2, 1, 3, 4, 5]:
        overall_folder = f'multiff_analysis/RL_models/SB3_stored_models/all_agents/env1_relu/ff{num_obs_ff}/'

        env_kwargs = {'num_obs_ff': num_obs_ff,
                    'max_in_memory_time': max_in_memory_time,
                    'print_ff_capture_incidents': False
                    #'reward_per_ff': 120,
        }

        # check if num_obs_ff is consistent with the name in overall_folder. If not, raise an error
        if not f'ff{env_kwargs["num_obs_ff"]}' in overall_folder:
            raise ValueError('num_obs_ff is not consistent with the name in overall_folder')

        gc.collect()
        print("Current parameters: ", params)

        # params = {'time_cost': 0.0, 'dv_cost_factor': 0.0, 'dw_cost_factor': 0.0, 'w_cost_factor': 0.0, 'v_noise_std': 0.0,
        # 'w_noise_std': 0.0}
        RLforFF = sb3_class.SB3forMultifirefly(**params, overall_folder=overall_folder,
                                                            **env_kwargs)

        #RLforFF.import_monkey_data(info_of_monkey, all_trial_features_m, pattern_frequencies_m, feature_statistics_m)
        RLforFF.streamline_everything(currentTrial_for_animation=None, num_trials_for_animation=None, duration=[10, 40],
                                        best_model_postcurriculum_exists_ok=False, to_load_latest_agent=False)

## get something from all the agents

In [ ]:
candidate_params = {"v_noise_std": [0, 0.1, 0.2, 0.3],
                    "w_noise_std": [0, 0.1, 0.2, 0.3],
                    "ffr_noise_scale": [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5],
                    "num_obs_ff": [2, 3],
                    "max_in_memory_time": [1, 2, 2.5]}
keys, values = zip(*candidate_params.items())
param_combinations = []
for bundle in product(*values):
    param_combinations.append(dict(zip(keys, bundle)))


temp_overall_folder = 'multiff_analysis/RL_models/SB3_stored_models/all_agents/gen_3/'
for count, params in enumerate(param_combinations):
    RLforFF = sb3_class.SB3forMultifirefly(**params, overall_folder=temp_overall_folder)
    RLforFF.make_env()
    RLforFF.make_agent()
    RLforFF.load_latest_agent()
    RLforFF.collect_data(n_steps=2000)
    super(type(RLforFF), RLforFF).make_or_retrieve_ff_dataframe(exists_ok=False)
    super(type(RLforFF), RLforFF).find_patterns()
    if len(RLforFF.ff_caught_T_new) < 1:
        print("No firefly was caught by the agent during testing. Rip") 
        continue
    RLforFF.set_animation_parameters(currentTrial=5, num_trials=3, k=1)
    RLforFF.call_animation_function(video_dir=RLforFF.overall_folder + 'all_videos', file_name=RLforFF.agent_id + '.mp4', plot_flash_on_ff=True)
    #RLforFF.combine_6_plots_for_neural_network()